In [583]:
import math

# 2.1 Echelon Form and Systematic Encoding Matrix

In [584]:
def read_matrix():
    matrix = open("H1.txt", "r").read().splitlines()
    for i in range(len(matrix)):
        matrix[i] = list(matrix[i])
        matrix[i] = [int(i) for i in matrix[i]]
    return matrix

def row_swap(matrix, i, j):
    matrix[i], matrix[j] = matrix[j], matrix[i]
    return matrix
    
def row_addtion(matrix, i, j):
    # R_i = R_i + R_j
    matrix[i] = [a ^ b for a, b in zip(matrix[i], matrix[j])]
    return matrix

def find_leading_one(matrix, row_i, column_i, k):
    while row_i <= k-1:
        if matrix[row_i][column_i] == 1:
            return row_i
        else:
            row_i += 1
    return -1

In [585]:
def echelon_and_generator_matrix(matrix):
    n = len(matrix[0])
    k = len(matrix)
    # Echelon
    for i in range(n-k):
        current_column_index = k+i
        # First find the row with leading one that fits the description of the identity matrix
        row_with_leading_one = find_leading_one(matrix, i, current_column_index, k)
        if row_with_leading_one == -1:
            print("Error: Cannot obtain echelon form")
        # Swap the row with leading one with the current top row
        matrix = row_swap(matrix, i, row_with_leading_one)
        # Remove 1s in rows other than current row
        for other_i in range(n-k):
            if other_i != i and matrix[other_i][current_column_index] == 1:
                matrix = row_addtion(matrix, other_i, i)
    echelon = matrix

    # Generator
    generator = []
    # Identity Matrix
    for i in range(k):
        row_i = []
        for j in range(k):
            if j == i:
                row_i.append(1)
            else:
                row_i.append(0)
        generator.append(row_i)
    # P^T
    for i in range(k):
        row_i = []
        for j in range(k):
            row_i.append(echelon[j][i])
        generator.append(row_i)  
        
    return echelon, generator

# 2.3 LDPC-Decoder

In [586]:
def read_y():
    f = open("y1.txt", "r").read()
    y = [int(i) for i in f]
    return y

def obtain_factor_graph(matrix):
    f = [[] for i in range(len(matrix))]
    for i in range(len(matrix)):
        for j in range(len(matrix[i])):
            if matrix[i][j] == 1:
                f[i].append(j)
    return f

def calculate_v2c(c2v_messages, v2c_messages, m_v, factor_graph):
    for i in range(len(m_v)):
        sum = m_v[i]
        for j in range(len(factor_graph)):
            if i in factor_graph[j]:
                sum += c2v_messages[j][i]
        for j in range(len(factor_graph)):
            if i in factor_graph[j]:
                v2c_messages[j][i] = sum - c2v_messages[j][i]
    return v2c_messages

def calculate_c2v(c2v_messages, v2c_messages, factor_graph):
    for i in range(len(c2v_messages)):
        # Get total product
        tanh_product = 1
        for f_i in factor_graph[i]:
            tanh_product *= math.tanh(v2c_messages[i][f_i]/2)
        # Remove individual variable from total product
        for f_i in factor_graph[i]:
            current_tan_product = tanh_product/math.tanh(v2c_messages[i][f_i]/2)
            if not math.isclose(current_tan_product, -1, abs_tol=1e-10) and not math.isclose(current_tan_product, 1, abs_tol=1e-10):
                c2v_messages[i][f_i] = 2*math.atanh(current_tan_product)
    return c2v_messages

def calculate_l(l, c2v_messages, m_v, factor_graph):
    for i in range(len(l)):
        sum_c2v = 0
        for j in range(len(c2v_messages)):
            if i in factor_graph[j]:
                sum_c2v += c2v_messages[j][i]
        l[i] = sum_c2v + m_v[i]
    return l

def parity_check(h, x):
    for row in h:
        sum = 0
        for i in range(len(row)):
            if row[i] == 1 and x[i] == 1:
                sum += 1
        if sum%2 != 0:
            return False
    return True

In [587]:
def ldpc_decoder(h, y, p, max_iter):
    # Factor graph
    factor_graph = obtain_factor_graph(h)
    # Code word
    x = y
    # Log-Likelihood Ratio
    llr = math.log((1-p)/p)
    # m_v is log-likelihood conditioned on observed value
    m_v = [llr if i == 0 else -llr for i in y]
    # l stands for log-likelihood list
    l = m_v
    # Declare variables
    v2c_messages = [[0 for j in range(len(h[0]))] for i in range(len(h))]
    c2v_messages = [[0 for j in range(len(h[0]))] for i in range(len(h))]
    # Set up v2c
    for i in range(len(v2c_messages)):
        for j in range(len(v2c_messages[i])):
            if j in factor_graph[i]:
                v2c_messages[i][j] = m_v[j]
    # Iteration
    for iteration in range(max_iter):
        # Update v2c
        if iteration != 0:
            v2c_messages = calculate_v2c(c2v_messages, v2c_messages, m_v, factor_graph)
            
        # Update c2v
        c2v_messages = calculate_c2v(c2v_messages, v2c_messages, factor_graph)
        
        # Belief Update
        l = calculate_l(l, c2v_messages, m_v, factor_graph)
        # print(l)
        
        # Decision Rule
        x = [0 if i >= 0 else 1 for i in l]
        # print(x)
        
        # Parity Check
        if parity_check(h, x):
            print("Iteration took to converge: "+str(iteration+1))
            return x, 0
        
    return x, -1

In [588]:
# Print Echelon and generator
h = read_matrix()
echelon, generator = echelon_and_generator_matrix(h)
print("Echelon")
for i in echelon:
    print(i)
print("Generator")
for i in generator:
    print(i)
    
# Decode
noise_ratio = 0.1
y = read_y()
max_iter = 20
print("Decoder")
print("Input")
print(y)
x, success = ldpc_decoder(echelon, y, noise_ratio, max_iter)
print("Decoded Message")
print(x)

Echelon
[1, 1, 1, 1, 0, 0]
[0, 1, 1, 0, 1, 0]
[1, 1, 0, 0, 0, 1]
Generator
[1, 0, 0]
[0, 1, 0]
[0, 0, 1]
[1, 0, 1]
[1, 1, 1]
[1, 1, 0]
Decoder
Input
[1, 0, 1, 0, 0, 1]
Iteration took to converge: 2
Decoded Message
[1, 0, 1, 0, 1, 1]


# 2.4 Recover Message